In [1]:
import time, joblib, os
import onnxruntime as ort
import numpy as np
import pandas as pd

from utils.dataset import Dataset
from tqdm.auto import tqdm

In [ ]:
pred_target_names = ['Pred_Load_Left_1(N)', 'Pred_Load_Left_2(N)', 'Pred_Load_Left_3(N)', 'Pred_Load_Left_4(N)', 'Pred_Load_Left_5(N)',
                     'Pred_Load_Right_1(N)', 'Pred_Load_Right_2(N)', 'Pred_Load_Right_3(N)', 'Pred_Load_Right_4(N)', 'Pred_Load_Right_5(N)']

In [42]:
absolute_error_names = ['Absolute_Error_Left_1(%)', 'Absolute_Error_Left_2(%)', 'Absolute_Error_Left_3(%)', 'Absolute_Error_Left_4(%)', 'Absolute_Error_Left_5(%)',
                        'Absolute_Error_Right_1(%)', 'Absolute_Error_Right_2(%)', 'Absolute_Error_Right_3(%)', 'Absolute_Error_Right_4(%)', 'Absolute_Error_Right_5(%)']

relative_error_names = ['Relative_Error_Left_1(%)', 'Relative_Error_Left_2(%)', 'Relative_Error_Left_3(%)', 'Relative_Error_Left_4(%)', 'Relative_Error_Left_5(%)',
                        'Relative_Error_Right_1(%)', 'Relative_Error_Right_2(%)', 'Relative_Error_Right_3(%)', 'Relative_Error_Right_4(%)', 'Relative_Error_Right_5(%)']

In [2]:
data_root_path = '.' + os.sep + 'data'
data_file_name_list = os.listdir(data_root_path)
data_file_name_list

['safe-boom-40-swing-180-load-40-.csv',
 'safe-boom-50-swing-180-load-50-.csv',
 'safe-boom-60-swing-180-load-60-.csv',
 'safe-boom-70-swing-180-load-100-.csv',
 'safe-boom-80-swing-180-load-120-.csv',
 'unsafe-swing-0-load-70-.csv',
 'unsafe-swing-0-load-90-.csv',
 'unsafe-swing-135-load-50-.csv',
 'unsafe-swing-135-load-70-.csv',
 'unsafe-swing-180-load-70-.csv',
 'unsafe-swing-180-load-90-.csv',
 'unsafe-swing-45-load-50-.csv',
 'unsafe-swing-45-load-70-.csv',
 'unsafe-swing-90-load-70-.csv',
 'unsafe-swing-90-load-90-.csv']

In [3]:
dataset_inst = Dataset(file_path_list=data_file_name_list)

  0%|          | 0/15 [00:00<?, ?it/s]

In [4]:
rf_model = joblib.load('rf_model.joblib')

In [12]:
val_feature = dataset_inst.get_val_dataset_for_model()[dataset_inst.get_model_feature_names()]
val_feature

,f10,f11,f12
0,66.879271,1.560000e-30,40.0
5,66.733915,9.370000e-31,40.0
10,66.585904,9.870000e-30,40.0
15,66.435284,3.480000e-30,40.0
20,66.282099,-4.470000e-31,40.0
...,...,...,...
60850,55.358831,9.000000e+01,90.0
60855,55.133860,9.000000e+01,90.0
60860,54.908862,9.000000e+01,90.0
60865,54.683882,9.000000e+01,90.0


In [31]:
rf_elapsed_time_list = []
rf_pred_list = []

for i in tqdm(range(val_feature.shape[0])):
    input_feature = pd.DataFrame(val_feature.iloc[i].values.reshape(1, -1), columns=dataset_inst.get_model_feature_names())
    t0 = time.time()
    rf_pred = rf_model.predict(input_feature)
    
    rf_elapsed_time_list.append(time.time() - t0)
    rf_pred_list.append(rf_pred)

  0%|          | 0/12175 [00:00<?, ?it/s]

In [35]:
np.vstack(rf_elapsed_time_list)

array([[0.02000499],
       [0.02911735],
       [0.03000808],
       ...,
       [0.01396012],
       [0.01500177],
       [0.01401901]])

In [39]:
rf_pred_arr = np.vstack(rf_pred_list)

In [40]:
val_dataset = dataset_inst.get_val_dataset()

In [47]:
actual_load = val_dataset[dataset_inst.get_data_target_names()].values
absolute_error = pd.DataFrame(abs(actual_load-rf_pred_arr), columns=absolute_error_names)
relative_error = pd.DataFrame((abs(actual_load-rf_pred_arr)/abs(actual_load))*100, columns=relative_error_names)
rf_result = pd.concat([val_dataset, pd.DataFrame(rf_pred_arr, columns=dataset_inst.get_data_target_names()) , absolute_error, relative_error], axis=1)

rf_result.describe()

,Boom_Angle(deg),Swing_Angle(deg),Load(Ton),Actual_Load_Left_1(N),Actual_Load_Left_2(N),Actual_Load_Left_3(N),Actual_Load_Left_4(N),Actual_Load_Left_5(N),Actual_Load_Right_1(N),Actual_Load_Right_2(N),...,Relative_Error_Left_1(%),Relative_Error_Left_2(%),Relative_Error_Left_3(%),Relative_Error_Left_4(%),Relative_Error_Left_5(%),Relative_Error_Right_1(%),Relative_Error_Right_2(%),Relative_Error_Right_3(%),Relative_Error_Right_4(%),Relative_Error_Right_5(%)
count,12175.000000,1.217500e+04,12175.000000,12175.000000,12175.000000,12175.000000,12175.000000,12175.000000,12175.000000,12175.000000,...,1.217500e+04,1.217500e+04,12175.000000,1.217500e+04,12175.000000,1.217500e+04,12175.000000,12175.000000,12175.000000,12175.000000
mean,63.520530,8.060977e+01,70.671047,280420.008592,270239.083296,258122.330067,245833.398274,235135.937374,208889.004659,195573.684626,...,4.573995e-01,3.401180e-01,0.285308,4.080203e-01,0.719929,6.513815e-01,0.474835,0.402675,0.569290,1.011804
std,11.369928,6.067786e+01,23.783388,110252.169601,71306.393845,49564.991274,74177.162316,113706.098285,109508.564461,67417.641354,...,1.062899e+00,7.732662e-01,0.632280,8.120523e-01,1.678449,1.486570e+00,1.018494,0.906481,1.309648,2.422191
min,40.000000,-3.180000e-28,40.000000,1351.487900,66400.622000,182590.050000,69831.394000,1380.030800,1038.630700,2564.975100,...,2.866178e-07,3.234412e-07,0.000003,5.340467e-07,0.000003,1.912578e-14,0.000000,0.000000,0.000000,0.000005
25%,59.460772,2.449026e+01,50.000000,205512.070000,214880.465000,214803.955000,191082.420000,147007.710000,115725.100000,144689.940000,...,1.510945e-02,9.118567e-03,0.004597,8.348353e-03,0.014252,1.961569e-02,0.012074,0.006672,0.013342,0.022736
50%,70.000000,7.574472e+01,70.000000,298398.750000,276248.910000,253963.040000,249802.650000,227857.440000,206677.930000,197477.600000,...,5.099143e-02,3.065399e-02,0.015767,3.162056e-02,0.061963,7.124407e-02,0.041213,0.022263,0.049097,0.096671
75%,70.000000,1.350000e+02,90.000000,358767.815000,314003.340000,283763.430000,297335.065000,327901.890000,294826.955000,238508.795000,...,2.897671e-01,2.383697e-01,0.188721,3.913114e-01,0.680518,5.424049e-01,0.409478,0.269952,0.390172,0.685086
max,80.000000,1.800000e+02,120.000000,493019.530000,470991.390000,474538.750000,477879.600000,489313.250000,492176.150000,392850.990000,...,2.057640e+01,1.289467e+01,10.577979,1.066342e+01,38.562538,4.932759e+01,25.158362,14.812926,17.679746,37.215117


In [23]:
onnx_model = ort.InferenceSession('rf_model.onnx')
input_name = onnx_model.get_inputs()[0].name
output_name = onnx_model.get_outputs()[0].name

In [24]:
input_data = val_feature.values.astype(np.float32)

In [44]:
onnx_elapsed_time = []
onnx_pred_list = []

for input_val in tqdm(input_data):
    input_val = input_val.reshape(1, -1)
    t0 = time.time()
    onnx_pred = onnx_model.run(output_names=None, input_feed={'float_input': input_val})[0][0]
    
    onnx_elapsed_time.append(time.time() - t0)
    onnx_pred_list.append(onnx_pred)

  0%|          | 0/12175 [00:00<?, ?it/s]

In [45]:
np.vstack(onnx_elapsed_time)

array([[0.0009985 ],
       [0.        ],
       [0.00099921],
       ...,
       [0.        ],
       [0.00099993],
       [0.        ]])

In [46]:
onnx_pred_arr = np.vstack(onnx_pred_list)

In [48]:
sum(onnx_elapsed_time)

3.02144718170166

In [49]:
actual_load = val_dataset[dataset_inst.get_data_target_names()].values
absolute_error = pd.DataFrame(abs(actual_load-onnx_pred_arr), columns=absolute_error_names)
relative_error = pd.DataFrame((abs(actual_load-onnx_pred_arr)/abs(actual_load))*100, columns=relative_error_names)
onnx_result = pd.concat([val_dataset, pd.DataFrame(onnx_pred_arr, columns=dataset_inst.get_data_target_names()) , absolute_error, relative_error], axis=1)

onnx_result.describe()

,Boom_Angle(deg),Swing_Angle(deg),Load(Ton),Actual_Load_Left_1(N),Actual_Load_Left_2(N),Actual_Load_Left_3(N),Actual_Load_Left_4(N),Actual_Load_Left_5(N),Actual_Load_Right_1(N),Actual_Load_Right_2(N),...,Relative_Error_Left_1(%),Relative_Error_Left_2(%),Relative_Error_Left_3(%),Relative_Error_Left_4(%),Relative_Error_Left_5(%),Relative_Error_Right_1(%),Relative_Error_Right_2(%),Relative_Error_Right_3(%),Relative_Error_Right_4(%),Relative_Error_Right_5(%)
count,12175.000000,1.217500e+04,12175.000000,12175.000000,12175.000000,12175.000000,12175.000000,12175.000000,12175.000000,12175.000000,...,1.217500e+04,1.217500e+04,1.217500e+04,12175.000000,12175.000000,12175.000000,1.217500e+04,1.217500e+04,12175.000000,1.217500e+04
mean,63.520530,8.060977e+01,70.671047,280420.008592,270239.083296,258122.330067,245833.398274,235135.937374,208889.004659,195573.684626,...,4.573995e-01,3.401180e-01,2.853080e-01,0.408020,0.719929,0.651382,4.748354e-01,4.026745e-01,0.569290,1.011804e+00
std,11.369928,6.067786e+01,23.783388,110252.169601,71306.393845,49564.991274,74177.162316,113706.098285,109508.564461,67417.641354,...,1.062899e+00,7.732662e-01,6.322804e-01,0.812052,1.678449,1.486570,1.018494e+00,9.064808e-01,1.309648,2.422191e+00
min,40.000000,-3.180000e-28,40.000000,1351.487900,66400.622000,182590.050000,69831.394000,1380.030800,1038.630700,2564.975100,...,2.599061e-07,4.043015e-07,6.748978e-07,0.000001,0.000001,0.000001,3.501496e-07,9.599949e-07,0.000002,8.647688e-07
25%,59.460772,2.449026e+01,50.000000,205512.070000,214880.465000,214803.955000,191082.420000,147007.710000,115725.100000,144689.940000,...,1.510576e-02,9.113316e-03,4.601936e-03,0.008342,0.014248,0.019615,1.207601e-02,6.676337e-03,0.013335,2.273499e-02
50%,70.000000,7.574472e+01,70.000000,298398.750000,276248.910000,253963.040000,249802.650000,227857.440000,206677.930000,197477.600000,...,5.099198e-02,3.065646e-02,1.576206e-02,0.031619,0.061963,0.071249,4.121672e-02,2.227294e-02,0.049090,9.667718e-02
75%,70.000000,1.350000e+02,90.000000,358767.815000,314003.340000,283763.430000,297335.065000,327901.890000,294826.955000,238508.795000,...,2.897743e-01,2.383658e-01,1.887202e-01,0.391307,0.680525,0.542402,4.094729e-01,2.699574e-01,0.390169,6.850759e-01
max,80.000000,1.800000e+02,120.000000,493019.530000,470991.390000,474538.750000,477879.600000,489313.250000,492176.150000,392850.990000,...,2.057641e+01,1.289467e+01,1.057798e+01,10.663422,38.562527,49.327596,2.515834e+01,1.481292e+01,17.679748,3.721512e+01
